In [1]:
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
import uproot

import numpy as np
np.seterr(divide='ignore', invalid='ignore', over='ignore')
import matplotlib.pyplot as plt
from FireHydrant.Tools.metfilter import MetFilters
from FireHydrant.Tools.correction import get_pu_weights_function, get_ttbar_weight, get_nlo_weight_function

# import pandas as pd
import awkward
from uproot_methods import TLorentzVectorArray
from FireHydrant.Tools.trigger import Triggers

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
from os.path import join
basedir='/uscms/home/wsi/nobackup/lpcdm/CMSSW_10_2_14_EGamma/src/Firefighter/ffLite/EventDisplayForLeptonJet/skimmedevents/SkimmedBkgMC'
datasets = {
    'DYJetsToLL_M-50': [join(basedir, 'DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8', 'ffNtuple.root'),],
    'QCD_Pt-120to170': [join(basedir, 'QCD_Pt-120to170_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'QCD_Pt-470to600': [join(basedir, 'QCD_Pt-470to600_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'QCD_Pt-50to80': [join(basedir, 'QCD_Pt-50to80_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'QCD_Pt-600to800': [join(basedir, 'QCD_Pt-600to800_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'QCD_Pt-800to1000': [join(basedir, 'QCD_Pt-800to1000_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'QCD_Pt-80to120': [join(basedir, 'QCD_Pt-80to120_MuEnrichedPt5_TuneCP5_13TeV_pythia8', 'ffNtuple.root'),],
    'TTJets': [join(basedir, 'TTJets_TuneCP5_13TeV-madgraphMLM-pythia8', 'ffNtuple.root'),],
}

In [2]:
class LeptonjetProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat('dataset', 'dataset')
        self._accumulator = processor.dict_accumulator({
            'run': processor.column_accumulator(np.zeros(shape=(0,))),
            'lumi': processor.column_accumulator(np.zeros(shape=(0,))),
            'event': processor.column_accumulator(np.zeros(shape=(0,))),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        output = self.accumulator.identity()
        if df.size==0: return output
        
        dataset = df['dataset']
        run = df['run']
        lumi = df['lumi']
        event = df['event']
        
        leptonjets = JaggedCandidateArray.candidatesfromcounts(
            df['pfjet_p4'],
            px=df['pfjet_p4.fCoordinates.fX'],
            py=df['pfjet_p4.fCoordinates.fY'],
            pz=df['pfjet_p4.fCoordinates.fZ'],
            energy=df['pfjet_p4.fCoordinates.fT'],
        )
        ljdautype = awkward.fromiter(df['pfjet_pfcand_type'])
        npfmu = (ljdautype==3).sum()
        ndsa = (ljdautype==8).sum()
        isegammajet = (npfmu==0)&(ndsa==0)
        ispfmujet = (npfmu>=2)&(ndsa==0)
        isdsajet = ndsa>0
        label = isegammajet.astype(int)*1+ispfmujet.astype(int)*2+isdsajet.astype(int)*3
        leptonjets.add_attributes(label=label)
        nmu = ((ljdautype==3)|(ljdautype==8)).sum()
        leptonjets.add_attributes(ismutype=(nmu>=2), iseltype=(nmu==0), isdsatype=(leptonjets.label==3))
                
        twoleptonjets = leptonjets.counts>=2
        dileptonjets = leptonjets[twoleptonjets]
        if dileptonjets.size==0: return output
        run=run[twoleptonjets]
        lumi=lumi[twoleptonjets]
        event=event[twoleptonjets]
        
        lj0 = dileptonjets[dileptonjets.pt.argmax()]
        lj1 = dileptonjets[dileptonjets.pt.argsort()[:, 1:2]]
        
        singleMuljEvents = dileptonjets.ismutype.sum()==1
        muljInLeading2Events = (lj0.ismutype | lj1.ismutype).flatten()
        channel_2mu2e = (singleMuljEvents&muljInLeading2Events).astype(int)*1
        
        doubleMuljEvents = dileptonjets.ismutype.sum()==2
        muljIsLeading2Events = (lj0.ismutype & lj1.ismutype).flatten()
        channel_4mu = (doubleMuljEvents&muljIsLeading2Events).astype(int)*2
        
        channel_ = channel_2mu2e + channel_4mu
        
        dsaIn4MuMask = ((lj0.isdsatype | lj1.isdsatype).flatten() & (channel_==2))
        
        output['run'] += processor.column_accumulator(run[dsaIn4MuMask])
        output['lumi'] += processor.column_accumulator(lumi[dsaIn4MuMask])
        output['event'] += processor.column_accumulator(event[dsaIn4MuMask])
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [5]:
outputs={}
for k in datasets:
    outputs[k] = processor.run_uproot_job({k: datasets[k]},
                                  treename='ffNtuplizer/ffNtuple',
                                  processor_instance=LeptonjetProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args=dict(workers=12, flatten=True),
                                  chunksize=500000,
                                 )

Processing: 100%|██████████| 1/1 [00:02<00:00,  2.00s/items]


In [6]:
for k, output in outputs.items():
    runs, lumis, events = output['run'].value.astype(int), output['lumi'].value.astype(int), output['event'].value.astype(int)
    print(f'\n{k}')
    if runs.size==0: continue
    for r,l,e in np.nditer([runs, lumis, events]):
        print('{}:{}:{}'.format(r, l, e))


DYJetsToLL_M-50
1:36746:83559096
1:44516:101227995
1:21752:49461879

QCD_Pt-120to170

QCD_Pt-470to600

QCD_Pt-50to80
1:4467:128130274

QCD_Pt-600to800

QCD_Pt-800to1000
1:16725:95359432

QCD_Pt-80to120

TTJets
